# Homework 3, Part I: Spark (25 points)


You are expected to run this notebook on Google Cloud.  Please make sure the Jupyter menu above shows "PySpark" at the upper right, and not "Python 3".  If it says "Python 3" then go to the *Kernel* menu and choose *PySpark*.

## Step 1: "Big Data": Transitive Closure over the Yelp Network

In the previous assignment, you had built several primitives, including breadth-first search, to compute over the Yelp social graph.  Some of your computations were limited by the amount of computational resources available.  We will now try to explore the broader graph.

Hints: For early testing of your solutions to Step 1, you should probably just load a subset of this file into `graph_sdf` to validate your solutions.  Later go back, add the contents of the other files to graph_sdf, and re-run your solutions.  Make sure you rerun your code with the full `graph_sdf` before submitting. 

### Step 1.1 Spark Initialization in Google DataProc

Spark is already pre-initialized in DataProc.  You'll want to import some of the libraries, though.

In [1]:
# YOUR CODE HERE
# raise NotImplementedError()

### Step 1.2 Loading the Data

Let’s read our social graph data from Yelp, which forms a directed graph.  Here, the set of nodes is also not specified; the assumption is that the only nodes that matter are linked to other nodes, and thus their IDs will appear in the set of edges.  For example, to load the file `input.txt` into a Spark DataFrame, you can use lines like the following. Note that we aren't running just `.txt` files in this assignment!

```
# Read lines from the text file, from Google Storage bucket my-bucket
input_sdf = spark.read.format("txt").load("gs://my-bucket/mypath/input.txt")
```

We’ll use the suffix `_sdf` to represent “Spark DataFrame,” much as we used `_sdf` to denote a Spark DataFrame in Homework 2. You will load a `yelp_reviews_sdf` table from the `yelp_reviews2.csv` file from your HW2. You may find dataset descriptions [here](https://www.kaggle.com/yelp-dataset/yelp-dataset/version/6#yelp_review.csv).

You'll need to upload this to Google Storage.  Uploading, and subsequently reading, this file may take a while. 

Add a function call (e.g., to `selectExpr()`) to rename the `user` and `business` nodes to `from_node` and `to_node`, to convert these to strings, respectively.

Please name the tables as we have described in this assignment. This helps with our grading and test cases.

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [8]:
# TODO: load yelp_reviews_sdf
# Worth 3 points in total
from pyspark.sql.functions import udf
spark = SparkSession.builder.appName('Graphs-HW3').getOrCreate()
try:
    yelp_reviews_sdf = spark.read.format('csv').option("header","true").load("yelp_review2.csv")
    
# YOUR CODE HERE
except:
    raise NotImplementedError()

In [9]:
# Just to show what it looks like
yelp_reviews_sdf.printSchema()


root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)



In [5]:
# This cell intentionally left blank ;-)


Now  create a `graph_sdf` and save it in the Spark TempView so that you can call your transitive closure function on it here, regardless of whether that function is written in SparkSQL or Spark DataFrame function calls.

In [21]:
# TODO: Convert from yelp_reviews_sdf to a graph
# Worth 2 points

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *

# YOUR CODE HERE
try:
    graph_sdf = yelp_reviews_sdf.select(col("user_id").alias("from_node"),col("business_id").alias("to_node"))
    graph_sdf.createOrReplaceTempView("graph")
    graph_sdf.show()
# YOUR CODE HERE

except:
    raise NotImplementedError()

graph_sdf.cache()
# graph_sdf.show()

+--------------------+--------------------+
|           from_node|             to_node|
+--------------------+--------------------+
|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|
|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|
|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|
|bv2nCi5Qv5vroFiqK...|ACFtxLv8pGrrxMm6E...|
|bv2nCi5Qv5vroFiqK...|s2I_Ni76bjJNK9yG6...|
|_4iMDXbXZ1p1ONG29...|8QWPlVQ6D-OExqXoa...|
|u0LXt3Uea_GidxRW1...|9_CGhHMz8698M9-Pk...|
|u0LXt3Uea_GidxRW1...|gkCorLgPyQLsptTHa...|
|u0LXt3Uea_GidxRW1...|5r6-G9C4YLbC7Ziz5...|
|u0LXt3Uea_GidxRW1...|fDF_o2JPU8BR1Gya-...|
|u0LXt3Uea_GidxRW1...|z8oIoCT1cXz7gZP5G...|
|u0LXt3Uea_GidxRW1...|XWTPNfskXoUL-Lf32...|
|u0LXt3Uea_GidxRW1...|13nKUHH-uEUXVZylg...|
|u0LXt3Uea_GidxRW1...|RtUvSWO_UZ8V3Wpj0...|
|u0LXt3Uea_GidxRW1...|Aov96CM4FZAXeZvKt...|
|u0LXt3Uea_GidxRW1...|0W4lkclzZThpx3V65...|
|u0LXt3Uea_GidxRW1...|fdnNZMk1NP7ZhL-YM...|
|u0LXt3Uea_GidxRW1...|PFPUMF38-lraKzLcT...|
|u0LXt3Uea_GidxRW1...|oWTn2IzrprsRkPfUL...|
|u0LXt3Uea_GidxRW1...|zgQHtqX0gq

DataFrame[from_node: string, to_node: string]

In [11]:
assert graph_sdf.count() > 0

### Step 1.3. Transitive Closure of the Graph

Now we would like to do the following: given a set of nodes, compute the set of all nodes reachable from these nodes.  This can be obtained via a type of transitive closure computation.

Define a function `transitive_closure(graph_sdf, origins_sdf, depth)` that returns a Spark DataFrame.

The result should be the set of all nodes from the `input graph_sdf` that are reachable via graph edges from the set of `origins_sdf`, in at most depth iterations (hops).  Both origins_sdf and the returned result should be DataFrames with a single attribute called node.  

You should treat the edges in the `graph_sdf` as undirected edges!  You should iterate until you have either hit the maximum depth or the set of newly discovered (frontier) nodes is empty.

Hints: this resembles your BFS algorithm from HW2, but you should take advantage of the opportunities to optimize.  Both the graph and the various node sets can easily have duplicates; you should make heavy use of duplicate removal (and repartition and cache) since you are only computing the set of reachable nodes. Also, to quickly check whether a DataFrame is empty, you can use something similar to the following. 

(You are not needed to update the following code block. Please start your code in the block containing `transitive_closure`

In [12]:
def sdf_is_empty(sdf):
    try:
        sdf.take(1)
        return False
    except:
        return True

Please insert your code for `transitive_closure` below.

In [22]:
# TODO: write the transitive_closure function
# Will be worth 10 points in total

from pyspark.sql.types import *
try:
# # # YOUR CODE HERE
    def transitive_closure(graph,origin,depth):
        schema = StructType([ StructField("node", StringType(), True) ])

        #Creating the full edge list
        G2 = graph
        G2 = G2.withColumnRenamed('from_node','x')
        G2 = G2.withColumnRenamed('to_node','from_node')
        G2 = G2.withColumnRenamed('x','to_node')
        G2 = G2.select("from_node",'to_node')
        G  = graph.union(G2)

        frontier = spark.createDataFrame([],schema)
        visited  = spark.createDataFrame([],schema)
        frontier = origin
        
        for iteration in range(0,depth+1):
            if sdf_is_empty(frontier) != True:
                #Select all nodes adjacent to frontier nodes
    #             G.repartition(10,'from_node')
                adj_G = frontier.join(G,frontier.node == G.from_node ,how='inner').select('to_node')
                adj_G = adj_G.join(visited,G.to_node == visited.node, how='left_outer').filter(F.col('node').isNull()).select('to_node').distinct()
                adj_G = adj_G.join(frontier,G.to_node == frontier.node, how='left_outer').filter(F.col('node').isNull()).select('to_node').distinct()

                #Move frontier to visited 
                visited = visited.union(frontier).cache()

                #Move adj to frontier
                frontier = adj_G.withColumnRenamed('to_node','node').dropDuplicates()
        # Return the visited nodes
        return(visited)

except:
    raise NotImplementedError()

### Step 1.4 Testing transitive closure

Compute a Spark DataFrame called `nodes_sdf` as follows:
* Select from the graph those `from_node` values matching "0", "1", "2", "3", or "4".
* Rename the column to `node`


In [14]:
# TODO: create nodes_sdf
# Worth 5 points

# YOUR CODE HERE
try:
    ids = ["0", "1", "2","3","4"]
    nodes_sdf = graph_sdf.filter(graph_sdf.from_node.isin(ids))
    nodes_sdf = ((nodes_sdf.withColumnRenamed("from_node","node")).drop(nodes_sdf.to_node))
    nodes_sdf = nodes_sdf.distinct()

# YOUR CODE HERE
except:
    raise NotImplementedError()

In [15]:
# You should only have one column
assert len(nodes_sdf.columns) == 1

print(nodes_sdf.count())

5


Compute a Spark DataFrame called `reachable_sdf` with the results of `transitive_closure(graph_sdf, nodes_sdf, 3)`

In [23]:
# YOUR CODE HERE
try:
    reachable_sdf = transitive_closure(graph_sdf, nodes_sdf, 3)
#     reachable_sdf = reachable_sdf.distinct()
#     reachable_sdf.show()
except:
    raise NotImplementedError()

In [20]:
## Test case that we get a result
assert reachable_sdf.take(1)[0].node != None

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


IndexError: list index out of range

Add and run two code Cells that call `count()` and then `show()`, respectively, on `reachable_sdf`

In [ ]:
reachable_sdf.count()

In [ ]:
reachable_sdf.show()